In [2]:
import yt_dlp as youtube_dl
from transformers import pipeline
from transformers.pipelines.audio_utils import ffmpeg_read
import tempfile
import os
from sentence_transformers import SentenceTransformer
import torch
import gradio as gr
import logging
import pandas as pd

In [ ]:
MODEL_NAME = "openai/whisper-large-v3"
BATCH_SIZE = 8
FILE_LIMIT_MB = 1000
YT_LENGTH_LIMIT_S = 3600  # limit to 1 hour YouTube files

device = 0 if torch.cuda.is_available() else "cpu"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

In [ ]:
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def _return_yt_html_embed(yt_url):
    video_id = yt_url.split("?v=")[-1]
    HTML_str = (
        f'<center> <iframe width="500" height="320" src="https://www.youtube.com/embed/{video_id}"> </iframe>'
        " </center>"
    )
    return HTML_str

def download_yt_audio(yt_url, filename):
    logging.info("Starting to download audio from YouTube.")
    try:
        info_loader = youtube_dl.YoutubeDL()
        info = info_loader.extract_info(yt_url, download=False)

    
        file_length = info["duration_string"]
        file_h_m_s = file_length.split(":")
        file_h_m_s = [int(sub_length) for sub_length in file_h_m_s]
        
        if len(file_h_m_s) == 1:
            file_h_m_s.insert(0, 0)
        if len(file_h_m_s) == 2:
            file_h_m_s.insert(0, 0)
        file_length_s = file_h_m_s[0] * 3600 + file_h_m_s[1] * 60 + file_h_m_s[2]
        
        if file_length_s > YT_LENGTH_LIMIT_S:
            yt_length_limit_hms = time.strftime("%H:%M:%S", time.gmtime(YT_LENGTH_LIMIT_S))
            file_length_hms = time.strftime("%H:%M:%S", time.gmtime(file_length_s))
            raise gr.Error(f"Maximum YouTube length is {yt_length_limit_hms}, got {file_length_hms} YouTube video.")
    
        ydl_opts = {"outtmpl": filename, "format": "bestaudio/best"}
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([yt_url])
        logging.info("Audio download completed.")
    except Exception as e:
        logging.error(f"Failed to download audio: {e}")
        raise


def yt_transcribe(yt_url, task, max_filesize=75.0):
    logging.info("Starting transcription process.")
    html_embed_str = _return_yt_html_embed(yt_url)
    try:
        with tempfile.TemporaryDirectory() as tmpdirname:
            filepath = os.path.join(tmpdirname, "video.mp4")
            download_yt_audio(yt_url, filepath)
            with open(filepath, "rb") as f:
                inputs = f.read()

        inputs = ffmpeg_read(inputs, pipe.feature_extractor.sampling_rate)
        inputs = {"array": inputs, "sampling_rate": pipe.feature_extractor.sampling_rate}
        
        logging.info("Starting transcription process.")
        text = pipe(inputs, batch_size=BATCH_SIZE, generate_kwargs={"task": task}, return_timestamps=True)["text"]
        logging.info("Transcription completed.")
        return html_embed_str, text
    except Exception as e:
        logging.error(f"Error during transcription: {e}")
        raise

In [ ]:
def launch(link):
    logging.info("Received new request for link: %s", link)
    try:
        html_embed_str, string = yt_transcribe(link, 'transcribe')
        logging.info("Embedding the transcription.")
        embedding = embedding_model.encode(string)

        df = pd.DataFrame([embedding])  # Create a DataFrame with one row of embeddings
        df.to_csv('embeddings.csv', mode='a', header=False, index=False)
        
        logging.info("Embedding completed.")
        return embedding
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise


iface = gr.Interface(launch,
                     inputs=gr.Textbox(),
                     outputs="text")

iface.launch()